In [8]:
import textract
import numpy as np
import scipy
import gensim
import os
import pandas as pd
import re
import math
from collections import Counter
from matplotlib import pyplot as plt
from gensim import corpora, models
%matplotlib inline

In [30]:
!pip install bertopic

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached numpy-1.21.4-cp37-cp37m-win_amd64.whl (14.0 MB)
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=c8264913e89661bf93f3a73282f3caf308756b2e7cf5c70abadda7b7c88dfa46
  Stored in directory: c:\users\juhász bálint\appdata\local\pip\cache\wheels\f9\8d\8d\f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=997c9c0d9dcb9d5df53d7edac450a41b3f68e3085b7666edc1190740c8dac4fc
  Stored in directory: c:\users\juhász bálint\appdata\local\pip\cache\wheels\90\f0\bb\ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
  Created wheel 

  ERROR: Command errored out with exit status 1:
   command: 'c:\anaconda\python.exe' 'c:\anaconda\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\JUHSZB~1\AppData\Local\Temp\tmpsmhika50'
       cwd: C:\Users\JUHSZB~1\AppData\Local\Temp\pip-install-zs84h4sc\hdbscan_6ba82894f60d44eeaa93d3ee0e84a87f
  Complete output (27 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\hdbscan
  copying hdbscan\flat.py -> build\lib.win-amd64-3.7\hdbscan
  copying hdbscan\hdbscan_.py -> build\lib.win-amd64-3.7\hdbscan
  copying hdbscan\plots.py -> build\lib.win-amd64-3.7\hdbscan
  copying hdbscan\prediction.py -> build\lib.win-amd64-3.7\hdbscan
  copying hdbscan\robust_single_linkage_.py -> build\lib.win-amd64-3.7\hdbscan
  copying hdbscan\validity.py -> build\lib.win-amd64-3.7\hdbscan
  copying hdbscan\__init__.py -> build\lib.win-amd64-3.7\hdbscan
  creating build\lib.win-amd

In [31]:
import bertopic

ModuleNotFoundError: No module named 'bertopic'

In [16]:
class FocusGroup:
    def __init__(self, filename):
        self.raw_text=str(textract.process('FocusGroups/' + filename + ".docx")).replace('b\'', '').replace('\'', '')
        
        self.parent_moderator_discussion=self.raw_text.split('\\n\\n\\n')[0].split('\\n\\n')
        self.within_moderator_discussion=self.raw_text.split('\\n\\n\\n')[1].split('\\n\\n')
        
        self.talkers_including_parents=np.array([self.parent_moderator_discussion[i].replace(':', '')
                                                for i in range(0, len(self.parent_moderator_discussion), 2)])
        self.text_including_parents=np.array([self.parent_moderator_discussion[i].replace(':', '')
                                                for i in range(1, len(self.parent_moderator_discussion), 2)])
        self.talkers_only_moderators=np.array([self.within_moderator_discussion[i].replace(':', '')
                                                for i in range(0, len(self.within_moderator_discussion), 2)])
        self.text_only_moderators=np.array([self.within_moderator_discussion[i].replace(':', '')
                                                for i in range(1, len(self.within_moderator_discussion), 2)])
        
        self.parent_list=[participant for participant in set(self.talkers_including_parents) if 'Parent' in participant]
        self.moderator_list=[participant for participant in set(self.talkers_including_parents) if 'Moderator' in participant]
        
        
    def get_participant_text(self, participant):
        if 'Parent' in participant:
            mask=[member==participant for member in self.talkers_including_parents]
            return list(self.text_including_parents[mask])
        elif 'Moderator' in participant:
            mask=[member==participant for member in self.talkers_including_parents]
            text_from_parent_discussion=self.text_including_parents[mask]
            mask=[member==participant for member in self.talkers_only_moderators]
            text_from_moderator_discussion=self.text_only_moderators[mask]
            return list(text_from_parent_discussion) + list(text_from_moderator_discussion)

In [28]:
focus_group=FocusGroup('Gaming_Group1')
participant='Parent 1'
focus_group.get_participant_text('Moderator 1')

['(silence)',
 '[inaudible 000246]',
 'Hi everyone. Welcome. Hi Parent 1, Parent 2, Parent 3. We\\re just going to wait for a couple more people to join.',
 '(silence)',
 '[inaudible 000343]',
 '(silence)',
 '[inaudible 000343].',
 'Hi [Administrator 000343], we\\re just waiting for everyone to join.',
 'Okay. Great. I think that Moderator 2 is joining also, so...',
 '(silence)',
 '[inaudible 000522]',
 'Hi Moderator 2.',
 'Hi Parent 4.',
 'Thank you everyone for joining, it looks like we\\re all on. So, thanks so much for taking out some time to have this focus group today. So, the purpose of the group is really just to get some more information [crosstalk 000550] about different family\\s experiences during COVID-19 as it relates to technology. I know you all filled out our crisis survey. So, this is really meant to go more in depth and get some qualitative information.',
 'So, I was thinking we could start by just going around and introducing ourselves. Maybe say the ages of your ki